# Init

In [1]:
%load_ext autoreload
%autoreload 2

####### INIT ######################################################################################

import datetime
today = datetime.date.today()

from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

## Imports ##
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees import augment_RM

augment_RM( robot )

%store -r

####### LIBS ######################################################################################

## Import Pytrees ##
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector

from assembly_trees import *

import math
from pmath import translate_pose , get_disance_between_poses , pose_components

from utils import is_matx_list, network_thermal_health
from pprint import pprint
from math import radians

if 0:
    print( "\n# Wrist Sensor Check #" )
    print( robot.ft.get_wrist_force() )
    print( robot.ft.bias_wrist_force() )
    sleep( 0.1 )
    print( robot.ft.get_wrist_force() )

print( "\n##!## READY :", today, "##!##" )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Setting up robot please wait...
my_components: {'hand_config': {'finger_length': 0.0415, 'module_name': 'hands.smarthand', 'finger_depth': 0.014, 'finger_width_outer': 0.015, 'finger_width_inner': 0.0, 'class_name': 'SmartHand'}, 'arm_config': {'default_joint_accel': 0.8, 'default_linear_speed': 0.1, 'default_joint_speed': 0.7, 'max_joint_accel': 1.4, 'max_joint_speed': 1.05, 'ip_address': '192.168.0.101', 'xmlrpc_port': '8003', 'max_linear_speed': 0.25, 'module_name': 'arms.ur5', 'default_linear_accel': 0.8, 'max_linear_accel': 1.2, 'class_name': 'UR5'}, 'cam_config': {'ci_cam_offset': [-0.036, -0.028, -0.079], 'pc_cam_offset': [-0.037, -0.033, -0.079], 'module_name': 'cameras.realsense', 'class_name': 'RealSense', 'camera_model': 'd410'}, 'ft_config': {'ip_address': '192.168.0.100', 'dataHz': 50.0, 'module_name': 'sensors.ftsensor_optoforce', 'max_workers': 10, 'r

# Global Vars

In [2]:
DATA_RATE  = 50
SLEEP_TIME = 10.0

# Spiral Insert Process

In [3]:
wheelPreInsertPose = wheelPostInsertPose.copy()
zApproach = 0.050
wheelPreInsertPose[2,3] += zApproach
print( "A pre-insert pose was defined." )

from py_trees.decorators import SuccessIsFailure
from py_trees.timers import Timer
from pathlib import Path
import operator
from random import random

statusKey = "insertStatus" # The name of the current logfile will go here
latchAll  = 0


def make_above_pose( trgtPose, vertOffset = 0.070 ):
    """ Return a pose that is vertically offset  """
    rtrnPose = trgtPose.copy()
    rtrnPose[2,3] += vertOffset
    return rtrnPose


def perturb_pose_radially_XY( pose, radMax ):
    """ Return a version of `pose` that is moved in the XY plane up to `radMax` distance """
    rtrnPose = pose.copy()
    dX = dY = 5000.0
    while np.linalg.norm( [dX, dY] ) > radMax:
        dX = -radMax + 2*radMax*random()
        dY = -radMax + 2*radMax*random()
    rtrnPose[0,3] += dX
    rtrnPose[1,3] += dY
    return rtrnPose


class SpiralRecoveryLoop_Simple( py_trees.composites.Sequence ):
    """ Tree for retrying Spiral Insertion until it succeeds, No monitoring """

    def setup( self ):
        """ Create a log file when the experiment begins """
        self.logPath = self.logPathGen()
        print( "Run log stored in" , self.logPath )
        
    def arrange_logs_and_write( self ):
        """ Gather everything that was logged, arrange in chrono order, and write to CSV """
        events = []
        for log in self.logs:
            events.extend( log.log )
            
         #  7. Define a timestamp comparison for sorting rows of data
        func = operator.itemgetter(0)
        def floatGetter(s):
            return float(func(s))

        #  8. Sort data 
        sortedEvents = sorted( events, key = floatGetter )
        
        with open( self.logPath, 'w', newline='') as csvfile:
            csvWriter = csv.writer( csvfile )
            csvWriter.writerows( sortedEvents )
            
        
    def __init__( self, Ntries, dwellTime = 10.0, ctrl = None,
                        perturbRad = 0.0 ):
        """ Create a tree """
        
        #### Setup ####
        
        # 0. Main loop should monitor continuously
        super().__init__( name = "Tilt Insert Loop", memory = 0 )
        
        self.zSAFE          = 0.100
        self.plungeDistance = 1.5 * self.zSAFE
        self.setdownForce   = 3.0
        self.descendSpeed   = 0.045
        self.biasWrist      = 1
        self.Ntries         = Ntries
        self.logs           = list()
        self.logPath        = None
        self.logPathGen     = get_outfile_namer( pathPrefix = "output/" , 
                                                 namePrefix = "GEAR-SPR-EVENT-LOG",
                                                 filePostfix = "txt" )
    
    
        #### Monitor Subtree ####
        
        recordBehav = Record_Classify( 
            recordFlagKey = "test_FLAG" , dataKey = "test_DATA" , labelKey = statusKey , 
            outFileGenFunc = get_outfile_namer( pathPrefix = "output/" , 
                                                namePrefix = "GEAR-SPR" ),
            recordHz = DATA_RATE , ctrl = ctrl 
        )
        
        self.add_child( recordBehav )
    
    
        #### Experiment Subtree ####
        
        ### Fetch Subtree ###
        fetchTree = Sequence( memory = 1 )
        
        print( dir( self ) ) 
        
        pickBgn = EventLogger( msgOrGen = "Pick:Begin", latch = latchAll )
        self.logs.append( pickBgn )
        
        #  1. Go to pickup pose
        goBehav = Jog_Safe( wheelGraspPose , 
                            zSAFE = self.zSAFE ,  
                            hover = 1 ,  
                            ctrl  = ctrl  )
    
        #  2. Pre-close around the gear
        closeBehav = Set_Fingers( openState = 0.045 , ctrl = ctrl )   

        #  3. Grasp gear
        graspBehav = Grasp_at_Pose( wheelGraspPose, 0.026 ,
                                    zApproach=0.050 , zClose=0.0 , 
                                    wdthNarrow = 0.050 , maxIter=5, zFree=0.100 , 
                                    ctrl = robot )
        
        behavPreInsertPose = wheelPreInsertPose.copy()
        
        if perturbRad > 0.0001:
            behavPreInsertPose = perturb_pose_radially_XY( wheelPreInsertPose, perturbRad )
        
        #  4. Goto pre-insertion pose
#         jogBehav = Jog_Safe( wheelPreInsertPose, 
        jogBehav = Jog_Safe( behavPreInsertPose, 
                             zSAFE = 0.100 ,  
                             hover = 0 ,  
                             ctrl  = robot  )
        
        pickEnd = EventLogger( msgOrGen = "Pick:End", latch = latchAll )
        self.logs.append( pickEnd )
        
        fetchTree.add_children( [ pickBgn, goBehav, closeBehav, graspBehav, jogBehav, pickEnd ] )
        
        
        ### Shortcut Subtree ### 
        shortTree = Selector( memory = 1 )
        shortSeq  = Sequence( memory = 1 )
        
        #  5. TODO: SHORTCUT HERE (Monitored Version Only)
        
        waitBehav = Timer( name="Wait for WRITE", duration=2.0)
        
        #  6. Sibling to the actual insertion subtree kicks off a thread for recording
        
        insertBgn = EventLogger( msgOrGen = "Insert:Begin", latch = latchAll )
        self.logs.append( insertBgn )
        
        
        stepFactor = 2.0

        #  7. Insertion behavior
        insertBehav = Spiral_Insert( 
#             wheelPreInsertPose ,
            behavPreInsertPose ,
            touch_force = 3.5 , drop_force = 2.0 , insert_force = 8.0 , 
            max_movement = zApproach + 0.030 , 
            lateralStopTorque = 0.800 , 
            pushbackF = 20.0 , spiralSpeed = 0.001 , descendSpeed = 0.00625 , biasWrist = 1 ,
            degrees_to_step = 3.25*2.0 , 
            maxAngle = 100 * 360 , startRadius = 0.00005 , 
            stepSize = 0.000025*1.75 , 
            max_radius = 0.010 ,
            posnMargin = 0.003 , orntMargin = 2.0/180 , 
            reliefStep_m = 0.0005 , reliefMargin_Nm = 0.050 , reliefN = 15 , 
            finalInsertHandZ = wheelPostInsertZ , 
            suppressDrop = 1 , suppressLateral = 0 , 
            recordFlagKey = "test_FLAG" , 
            ctrl = robot,
            chaseMode = 1, chaseStep = 0.00010 , N_chaseMax = 10 
        )   
        
        insertStatus = Store_Subtree_Status( insertBehav, statusKey )
        
        insertEnd = EventLogger( msgOrGen = "Insert:End", latch = latchAll )
        self.logs.append( insertEnd )
        
        shortSeq.add_children( [ waitBehav, insertBgn, insertStatus, insertEnd ] )
        shortTree.add_children( [ shortSeq, insertEnd ] )
        
        ### Reset Subtree ###
        resetTree = Sequence( memory = 1 )
        resetBgn  = EventLogger( msgOrGen = "Reset:Begin", latch = latchAll )
        self.logs.append( resetBgn )
        
        #  8. Goto pickup pose "goBehav" (re-used)
        
        #  9. Move the shaft gently to the table
        placeBehav = Move_to_Contact( relMove = [ 0 , 0 , -self.plungeDistance ] , 
                                      Fmag = self.setdownForce , speed = self.descendSpeed , 
                                      accel = 0.17,
                                      biasWrist = self.biasWrist , ctrl = ctrl )
        
        #  9. Drop, whether we won or not
        dropBehav = Set_Fingers( openState = 1.0 , ctrl = ctrl )
        
        resetEnd = EventLogger( msgOrGen = "Reset:End", latch = latchAll )
        self.logs.append( resetEnd )
        
        
        # 10. Construct the reset subtree
        resetTree.add_children( [resetBgn, goBehav, placeBehav, dropBehav, resetEnd] )
        
         #### Main Retry Loop ####
        
        #  0. Create the main MC driver
        experimentTree = py_trees.composites.Sequence( memory = 1 )
        experimentRoot = py_trees.composites.Selector( memory = 1 )
        
        
        #  1. Add the decorator
        retryLoop = Run_to_X_Failures_DECO(  
            experimentRoot ,
            X_allowedFails = Ntries
        )
        
        ### Success Subtree ###
        successTree = Sequence( memory = 1 )
        successTree.add_children( [goBehav, placeBehav, dropBehav] )
        
        # We can only reach the successtree after winning the retryLoop
        experimentTree.add_children( [retryLoop, successTree] )
        self.add_child( experimentTree )

        # 11. Add the record/insert/reset siblings as children
        experimentRoot.add_children( [
            SuccessIsFailure( fetchTree ), 
            shortSeq,
            SuccessIsFailure( resetTree ),
        ] )

print( "Spiral BT Created!" )

A pre-insert pose was defined.
Spiral BT Created!


# Spiral Insert: Recording Pipeline

In [4]:
N        =  2
N_retry  =  1 # Number of max tries per trial (including original)
stpSleep =  8
divSleep = 30
div      =  5
j        =  1
checks   = list()
chkHosts  = ['192.168.0.102', '192.168.0.100'] # Ping PC+FT to gauge net health
ft_host   = chkHosts[1]

from time import sleep

for i in range( 1 , N+1 ):
    
    try:
        
        print( "Iteration" , i , "of" , N )
        
        # 6. Sleep for human reset
        if i > 1:
            sleep( stpSleep )

        # 3. Release
        robot.hand.release()
        
        # 1. Reset tree

        # Only perturb for half of the trials
        if i%2==1:
            nudge = 0.002
#             nudge = 0.004
        else:
            nudge = 0.000
        print( "\nUsing nudge:", nudge, '\n' )
        
        rootNode = SpiralRecoveryLoop_Simple( Ntries = N_retry, ctrl = robot, perturbRad = nudge )
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode, N = 5000, tickPause = 0.25, 
                           breakOnFailure = 1, breakOnSuccess = 1, Nverb = 1000 )
        
        # 3. Release
        robot.hand.set_finger_width( 0.050 ) 
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )
        
        rootNode.arrange_logs_and_write()
        
        checks.append( 
            network_thermal_health( remoteHosts=chkHosts ) # Ping optoforce raspi
        )
        
        if i%div==0:
            sleep( divSleep )
            
        rootNode = None

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break
        
#     if i > N:
#         break
    

for chk in checks:
    pprint( chk )

Iteration 1 of 2

Using nudge: 0.002 

Connected to http://localhost:20000
['Ntries', '_DEBUG', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_child', 'add_children', 'arrange_logs_and_write', 'attach_blackboard_client', 'biasWrist', 'blackboards', 'blackbox_level', 'children', 'current_child', 'current_index', 'descendSpeed', 'feedback_message', 'has_parent_with_instance_type', 'has_parent_with_name', 'id', 'initialise', 'insert_child', 'iterate', 'iterator', 'logPath', 'logPathGen', 'logger', 'logs', 'memory', 'name', 'parent', 'plungeDistance', 'prepend_child', 'qualified_name', 'remove_all_children', 'remove_child', 'remove_child_by_id', 'replace_child', 'setdownForce', 'setup', 'setup_subtree', 'setup_wit

# Tilt Insert: Shaft Recording

In [ ]:
from py_trees.decorators import SuccessIsFailure
from py_trees.timers import Timer
from pathlib import Path
import operator

statusKey = "insertStatus" # The name of the current logfile will go here
latchAll  = 0

# ASMBB.set( eventKey, "test.txt" )

# class TiltRecoveryLoop_Simple( Run_to_X_Failures_DECO ):
class TiltRecoveryLoop_Simple( py_trees.composites.Sequence ):
    """ Tree for retrying Spiral Insertion until it succeeds, No monitoring """
    
    def setup( self ):
        """ Create a log file when the experiment begins """
        self.logPath = self.logPathGen()
        print( "Run log stored in" , self.logPath )
        
    def arrange_logs_and_write( self ):
        """ Gather everything that was logged, arrange in chrono order, and write to CSV """
        events = []
        for log in self.logs:
            events.extend( log.log )
            
         #  7. Define a timestamp comparison for sorting rows of data
        func = operator.itemgetter(0)
        def floatGetter(s):
            return float(func(s))

        #  8. Sort data 
        sortedEvents = sorted( events, key = floatGetter )
        
        with open( self.logPath, 'w', newline='') as csvfile:
            csvWriter = csv.writer( csvfile )
            csvWriter.writerows( sortedEvents )
    
    def __init__( self, Ntries, dwellTime = 10.0, ctrl = None ):
        """ Create a tree """
        
        #### Setup ####
        
        # 0. Main loop should monitor continuously
        super().__init__( name = "Tilt Insert Loop", memory = 0 )
        
        self.zSAFE          = 0.100
        self.plungeDistance = 1.5 * self.zSAFE
        self.setdownForce   = 1.0
        self.descendSpeed   = 0.045
        self.biasWrist      = 1
        self.Ntries         = Ntries
        self.logs           = list()
        self.logPath        = None
        self.logPathGen     = get_outfile_namer( pathPrefix = "output/" , 
                                                 namePrefix = "SHAFT-TLT-EVENT-LOG",
                                                 filePostfix = "txt" )
        
        
        #### Monitor Subtree ####
        
        recordBehav = Record_Classify( 
            recordFlagKey = "test_FLAG" , dataKey = "test_DATA" , labelKey = statusKey , 
            outFileGenFunc = get_outfile_namer( pathPrefix = "output/" , 
                                                    namePrefix = "SHAFT-TLT" ),
            recordHz = DATA_RATE , ctrl = ctrl 
        )
        
        self.add_child( recordBehav )
        
        
        #### Experiment Subtree ####
        
        ### Fetch Subtree ###
        fetchTree = Sequence( memory = 1 )
        
        print( dir( self ) ) 
        
        pickBgn = EventLogger( msgOrGen = "Pick:Begin", latch = latchAll )
        self.logs.append( pickBgn )
        
        #  1. Go to pickup pose
        goBehav = Jog_Safe( shaftGraspPose , 
                            zSAFE = self.zSAFE ,  
                            hover = 1 ,  
                            ctrl  = ctrl  )

        #  2. Pre-close around the shaft
        closeBehav = Set_Fingers( openState = 0.040 , ctrl = ctrl )   

        #  3. Grasp shaft
        graspBehav = Grasp_at_Pose( shaftGraspPose, 0.015 ,
                                    zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.030 , maxIter=5, zFree=0.100 , 
                                    ctrl = ctrl )

        #  4. Goto pre-insertion pose
        jogBehav = Jog_Safe( shaftPreInsertPose , 
                             zSAFE = self.zSAFE ,  
                             hover = 0 ,  
                             ctrl  = ctrl  )
        
        pickEnd = EventLogger( msgOrGen = "Pick:End", latch = latchAll )
        self.logs.append( pickEnd )
        
        fetchTree.add_children( [ pickBgn, goBehav, closeBehav, graspBehav, jogBehav, pickEnd ] )
        
        
        ### Shortcut Subtree ### 
        shortTree = Selector( memory = 1 )
        shortSeq  = Sequence( memory = 1 )
        
        #  5. TODO: SHORTCUT HERE (Monitored Version Only)
        
        waitBehav = Timer( name="Wait for WRITE", duration=2.0)
        
        #  6. Sibling to the actual insertion subtree kicks off a thread for recording
        
        insertBgn = EventLogger( msgOrGen = "Insert:Begin", latch = latchAll )
        self.logs.append( insertBgn )
        
        
        #  7. Insertion behavior
        tiltBehav = Tilt_Insert( shaftPreInsertPose ,
                                 tilt_angle_deg = 7.25 , part_offset = 0.016 , dia = 0.015 , 
                                 touch_force = 0.6 , insert_force = 1.5 , max_movement = 0.045 , 
                                 biasWrist = 1 , offset_dir = np.array( [ 1.0, 0.0, 0.0 ] ), 
                                 posnMargin = 0.003 , orntMargin = 2.0/180 , 
                                 finalInsertHandZ = fatPegFullInsertZ , 
                                 recordFlagKey = "test_FLAG" ,
                                 dropSpeed = 0.0313 , ctrl = ctrl )
        tiltStatus = Store_Subtree_Status( tiltBehav, statusKey )
        
        insertEnd = EventLogger( msgOrGen = "Insert:End", latch = latchAll )
        self.logs.append( insertEnd )
        
        
#         shortSeq.add_children( [waitBehav, insertBgn, tiltBehav, insertEnd] )
        shortSeq.add_children( [waitBehav, insertBgn, tiltStatus, insertEnd] )
#         shortSeq.add_children( [tiltBehav, waitBehav, ] )
#         shortSeq.add_children( [recordBehav, tiltBehav] )
        shortTree.add_children( [ shortSeq, insertEnd ] )
        
        ### Reset Subtree ###
        resetTree = Sequence( memory = 1 )
        
        resetBgn = EventLogger( msgOrGen = "Reset:Begin", latch = latchAll )
        self.logs.append( resetBgn )
        
        
        #  8. Goto pickup pose "goBehav"
        
        #  9. Move the shaft gently to the table
        placeBehav = Move_to_Contact( relMove = [ 0 , 0 , -self.plungeDistance ] , 
                                      Fmag = self.setdownForce , speed = self.descendSpeed , 
                                      accel = 0.17,
                                      biasWrist = self.biasWrist , ctrl = ctrl )
        
        
        #  9. Drop, whether we won or not
        dropBehav = Set_Fingers( openState = 1.0 , ctrl = ctrl )
        
        resetEnd = EventLogger( msgOrGen = "Reset:End", latch = latchAll )
        self.logs.append( resetEnd )
        
        
        # 10. Construct the reset subtree
        resetTree.add_children( [resetBgn, goBehav, placeBehav, dropBehav, resetEnd] )
        
        
        #### Main Retry Loop ####
        
        #  0. Create the main MC driver
        experimentTree = py_trees.composites.Sequence( memory = 1 )
        experimentRoot = py_trees.composites.Selector( memory = 1 )
        
        
        #  1. Add the decorator
        retryLoop = Run_to_X_Failures_DECO(  
            experimentRoot ,
            X_allowedFails = Ntries
        )
        
        ### Success Subtree ###
        successTree = Sequence( memory = 1 )
        successTree.add_children( [goBehav, placeBehav, dropBehav] )
        
        # We can only reach the successtree after winning the retryLoop
        experimentTree.add_children( [retryLoop, successTree] )
        self.add_child( experimentTree )

        # 11. Add the record/insert/reset siblings as children
        experimentRoot.add_children( [
            SuccessIsFailure( fetchTree ), 
#             shortSeq, 
#             SuccessIsFailure( shortSeq ), 
            shortSeq,
            SuccessIsFailure( resetTree ),
        ] )
        
        
        
    

# Tilt Insert Pipeline

In [4]:
N        =  3
divSleep =  5
t_sleep  =  5
j        =  1

N_sleep = int( N / divSleep )

from time import sleep

for i in range( 1 , N+1 ):
    
    try:
        
        print( "Iteration" , i , "of" , N )
        
        # 6. Sleep for human reset
        if i > 1:
            sleep( divSleep )
        
        # 1. Reset tree
#         rootNode = get_shaft_insert_tree()
        rootNode = TiltRecoveryLoop_Simple( Ntries = 3, ctrl = robot )
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode, N = 1000, tickPause = 0.25, 
                           breakOnFailure = 1, breakOnSuccess = 1, Nverb = 0 )
        
        # 3. Release
        robot.hand.release()
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )
            
        rootNode.arrange_logs_and_write()

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break

Iteration 1 of 3
['Ntries', '_DEBUG', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_child', 'add_children', 'arrange_logs_and_write', 'attach_blackboard_client', 'biasWrist', 'blackboards', 'blackbox_level', 'children', 'current_child', 'current_index', 'descendSpeed', 'feedback_message', 'has_parent_with_instance_type', 'has_parent_with_name', 'id', 'initialise', 'insert_child', 'iterate', 'iterator', 'logPath', 'logPathGen', 'logger', 'logs', 'memory', 'name', 'parent', 'plungeDistance', 'prepend_child', 'qualified_name', 'remove_all_children', 'remove_child', 'remove_child_by_id', 'replace_child', 'setdownForce', 'setup', 'setup_subtree', 'setup_with_descendants', 'shutdown', 'status', 'stop', 'terminate',

# Common Functions

In [5]:
robot.arm.align_tcp( )

True

In [19]:
robot.hand.grip()

True

In [ ]:
robot.hand.release()

In [6]:
robot.hand.set_finger_width( 0.040 ) 

True

In [ ]:
robot.arm.move( wheelGraspPose )

In [ ]:
print( robot.arm.get_tcp_pose() )
print( robot.arm.get_tcp_pose()[2,3] )
print( baseTouchPoseZ )

In [ ]:
robot.ft.get_wrist_force()

In [ ]:
robot.hand.deactivate_motors()

In [ ]:
robot.hand.zero_fingers()

# Setup

## Behavior: Move to Contact

In [15]:
rootNode =  Move_to_Contact( Fmag = 2.5 , relMove = [0.000, 0.000, -0.100] , biasWrist = 1 , 
                         mode = 'l' , speed = 0.0625 , accel = 0.17 , ctrl = robot )
print( "FT before the behavior:" , robot.ft.get_wrist_force() )
run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 )
print( "FT After the behavior:" , robot.ft.get_wrist_force() )

FT before the behavior: [-0.7564454078674316, 4.411328315734863, -1.97247314453125, -0.35438072681427, -0.0849374532699585, -0.12160936743021011]
Root node Move_to_Contact succeeded!


Simulation completed! with status rootNode.status 


FT After the behavior: [0.8244140148162842, -0.09648436307907104, -13.484954833984375, 0.025869131088256836, 0.23348236083984375, -0.008314445614814758]


## Behavior: Relieve Wrist Torque

In [18]:
rootNode =  Minimize_Wrist_Torque_XY( numTrials = 30 , marginT=0.05 , moveStep=0.0005  , ctrl = robot )
print( "FT before the behavior:" , robot.ft.get_wrist_force() )
run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 )
print( "FT After the behavior:" , robot.ft.get_wrist_force() )


FT before the behavior: [0.22441411018371582, -0.2964843511581421, -11.6849365234375, 0.04186904430389404, 0.12548232078552246, -0.005314454436302185]
Root node Minimize_Wrist_Torque_XY succeeded!


Simulation completed! with status rootNode.status 


FT After the behavior: [-1.375585913658142, -0.396484375, -11.6849365234375, 0.04386913776397705, -0.11851763725280762, -0.00931444764137268]


## Setup Actions

In [2]:
robot.arm.move( wheelGraspPose )

In [5]:
robot.arm.move( wheelPreInsertPose )

In [5]:
robot.arm.align_tcp()

True

In [7]:
robot.hand.set_finger_width( 0.040 ) 

True

In [14]:
robot.hand.set_finger_width( 0.0 ) 

True

In [11]:
robot.hand.set_finger_torque( 0.2 ) 

True

In [13]:
robot.hand.set_finger_torque( 1 ) 

True

In [8]:
robot.hand.grip()

True

In [4]:
print( dir( robot.arm ) )

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'align_tcp', 'arm_default_joint_accel', 'arm_default_joint_speed', 'arm_default_linear_accel', 'arm_default_linear_speed', 'arm_max_joint_accel', 'arm_max_joint_speed', 'arm_max_linear_accel', 'arm_max_linear_speed', 'brake_release', 'common_poses', 'connected', 'disable_freedrive', 'dummy_stop', 'enable_freedrive', 'get_joint_angles', 'get_safety_status_bits', 'get_status_bits', 'get_tcp_force', 'get_tcp_pose', 'get_tcp_pose_vec', 'kill_rtde', 'move', 'move_speed', 'move_timed', 'move_to_common_pose', 'power_off', 'power_on', 'proxy', 'robot_arm_ip', 'rotate_tcp', 'rtde_process', 's', 's_dashboard', 'send_command_to_robot', 'send_script_to_robot', 'set_joint_angles',

### Gear Poses

In [9]:
# wheelGraspPose = robot.-

Stored 'wheelGraspPose' (ndarray)


In [10]:
# wheelPostInsertPose = robot.arm.get_tcp_pose()
# wheelPostInsertZ = wheelPostInsertPose[2,3]
# %store wheelPostInsertPose
# %store wheelPostInsertZ

In [7]:
# wheelPostInsertPose[0,3] += 0.001
# wheelPostInsertPose[1,3] += 0.002
# %store wheelPostInsertPose

Stored 'wheelPostInsertPose' (ndarray)


In [8]:
# wheelPreInsertPose = wheelPostInsertPose.copy()
# zApproach = 0.070
# wheelPreInsertPose[2,3] += zApproach
# %store wheelPreInsertPose

Stored 'wheelPreInsertPose' (ndarray)


In [ ]:
# wheelPreOffsetYPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetYPose

In [ ]:
# wheelPreOffsetXPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetXPose

### Shaft Poses

In [25]:
# shaftGraspPose = robot.arm.get_tcp_pose()
# %store shaftGraspPose

Stored 'shaftGraspPose' (ndarray)


In [26]:
# shaftPreInsertPose = robot.arm.get_tcp_pose()
# %store shaftPreInsertPose

Stored 'shaftPreInsertPose' (ndarray)


### General Poses

In [6]:
# safeCenterPose = robot.arm.get_tcp_pose()
# %store safeCenterPose

Stored 'safeCenterPose' (ndarray)


In [27]:
# baseTouchPose  = robot.arm.get_tcp_pose()
# baseTouchPoseZ = baseTouchPose[2,3]
# %store baseTouchPose
# %store baseTouchPoseZ

Stored 'baseTouchPose' (ndarray)
Stored 'baseTouchPoseZ' (float64)
